## EXTRACCIÓN DE CARACTERÍSTICAS
Este notebook extrae glosarios para las distintas categorías de texto utilizando la librería gensim.

In [1]:
import pandas as pd
import gensim
from gensim import corpora
from gensim import models
from gensim.utils import simple_preprocess

Cargamos nuestro dataframe de noticias

In [2]:
noticias_dataframe = pd.read_csv('../Datos/noticias.csv')

Separamos nuestras noticias por categoría

In [3]:
deportes_dataframe = noticias_dataframe[noticias_dataframe["category"]=="sports"]
deportes_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 59
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  60 non-null     int64 
 1   index       60 non-null     int64 
 2   category    60 non-null     object
 3   n_doc       60 non-null     int64 
 4   title       60 non-null     object
 5   path        60 non-null     object
 6   link        60 non-null     object
 7   docs        60 non-null     object
 8   corpus      60 non-null     object
dtypes: int64(3), object(6)
memory usage: 4.7+ KB


In [4]:
salud_dataframe = noticias_dataframe[noticias_dataframe["category"]=="health"]
salud_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 60 to 119
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  60 non-null     int64 
 1   index       60 non-null     int64 
 2   category    60 non-null     object
 3   n_doc       60 non-null     int64 
 4   title       60 non-null     object
 5   path        60 non-null     object
 6   link        60 non-null     object
 7   docs        60 non-null     object
 8   corpus      60 non-null     object
dtypes: int64(3), object(6)
memory usage: 4.7+ KB


In [5]:
ciencia_dataframe = noticias_dataframe[noticias_dataframe["category"]=="science"]
deportes_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 59
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  60 non-null     int64 
 1   index       60 non-null     int64 
 2   category    60 non-null     object
 3   n_doc       60 non-null     int64 
 4   title       60 non-null     object
 5   path        60 non-null     object
 6   link        60 non-null     object
 7   docs        60 non-null     object
 8   corpus      60 non-null     object
dtypes: int64(3), object(6)
memory usage: 4.7+ KB


In [6]:
politica_dataframe = noticias_dataframe[noticias_dataframe["category"]=="politics"]
politica_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 180 to 239
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  60 non-null     int64 
 1   index       60 non-null     int64 
 2   category    60 non-null     object
 3   n_doc       60 non-null     int64 
 4   title       60 non-null     object
 5   path        60 non-null     object
 6   link        60 non-null     object
 7   docs        60 non-null     object
 8   corpus      60 non-null     object
dtypes: int64(3), object(6)
memory usage: 4.7+ KB


### Extracción de glosario mediante TFIDF


La forma más sencilla para obtener nuestro glosario es mediante el esquema *tf-idf*

In [35]:
def extrae_glosario_tf_idf(categoria_dataframe, size):
    doc_tokens = [simple_preprocess(corpus) for corpus in categoria_dataframe["corpus"]]
    dictionary = corpora.Dictionary(doc_tokens)
    bow_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in doc_tokens]
    tfidf = models.TfidfModel(bow_corpus)
    bow_tfidf = tfidf[bow_corpus]
    tfidf_dic = {dictionary.get(id): value for doc in bow_tfidf for id, value in doc}
    tfidf_list = [k for k, v in sorted(tfidf_dic.items(), key=lambda item: item[1], reverse = True)]
    return tfidf_list[:size]
    

In [36]:
extrae_glosario_tf_idf(salud_dataframe, 30)

['arterial',
 'cafa',
 'epilepsia',
 'cabello',
 'folcodina',
 'cannabis',
 'antidepresivo',
 'zumo',
 'genital',
 'strep',
 'dash',
 'entrenar',
 'olor',
 'verruga',
 'huevo',
 'carbohidrato',
 'creatividad',
 'marfan',
 'presion',
 'vision',
 'vih',
 'pulmonar',
 'espacial',
 'mascarilla',
 'lobo',
 'memoria',
 'alga',
 'wakame',
 'pet',
 'persistente']

In [37]:
extrae_glosario_tf_idf(deportes_dataframe, 30)

['kyrgios',
 'mbappe',
 'exencion',
 'warren',
 'stakhovsky',
 'falso',
 'jamas',
 'smash',
 'tenerife',
 'nets',
 'reserva',
 'gaal',
 'formato',
 'campazzo',
 'ferrero',
 'lebron',
 'hernanga',
 'mez',
 'magnussen',
 'steiner',
 'horford',
 'domenicali',
 'marko',
 'seguidor',
 'gasto',
 'cuento',
 'gonzalez',
 'gonzalo',
 'pts',
 'reb']

In [38]:
extrae_glosario_tf_idf(politica_dataframe, 30)

['silva',
 'larsen',
 'ucraniano',
 'ex',
 'secreto',
 'sinema',
 'mexicano',
 'torra',
 'esparza',
 'insulto',
 'benito',
 'mocia',
 'almeida',
 'cni',
 'nicolas',
 'profesional',
 'disponible',
 'bolaaos',
 'feija',
 'demarcacia',
 'juarez',
 'obrador',
 'autoritariaa',
 'maximo',
 'carlos',
 'junts',
 'rufian',
 'escuchas',
 'mitsotakis',
 'juan']

In [39]:
extrae_glosario_tf_idf(ciencia_dataframe, 30)

['sal',
 'vacuna',
 'llama',
 'meteoro',
 'apple',
 'honor',
 'gripe',
 'llnl',
 'quipus',
 'ska',
 'lluvia',
 'hidra',
 'navidad',
 'banyoles',
 'neandertal',
 'programador',
 'bankman',
 'fried',
 'tv',
 'mental',
 'trastorno',
 'gemanidas',
 'mutacion',
 'fusia',
 'captura',
 'geno',
 'izquierdo',
 'leo',
 'regalo',
 'congelacia']